In [40]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options #Helps dictate what to feature when pulling up whatever platform browser
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.request
import re
import numpy as np
import os
from jinja2 import Template


PATH = r'C:\Program Files (x86)\chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument("--window-size=1920,1080");
options.add_argument('start-maximized')
options.add_argument('--headless')
options.add_argument('disable-infobars')
driver = webdriver.Chrome(options = options, executable_path = PATH)
wait = WebDriverWait(driver,10)

attackset = pd.DataFrame(columns = ['Player Name','Season', 'Goals','Goals Per Match','Headed Goals',
                    'Goals with Right Foot','Goals with Left Foot','Penalties Scored', 'Freekicks Scored', 'Shots','Shots on Target',
                    'Shooting Accuracy', 'Hit Woodwork', 'Big Chances Missed'])

defenceset = pd.DataFrame(columns = ['Player Name','Season', 'Clean Sheets','Goals Conceded','Tackles','Tackle Success %',
                    'Last Man Tackles','Blocked Shots','Interceptions', 'Clearances', 'Headed Clearances','Clearances Off Line',
                    'Recoveries','Duels Won','Duels Lost', "SuccessFul 50/50's", 'Aerial Battles Won','Aerial Battles Lost', 'Own Goals','Errors Leading to Goal'])
Positionset = pd.DataFrame(columns = ['Player Name', 'Position'])
BaseInfo = pd.DataFrame(columns= ['Player Name', 'Seasons', 'Team'])
FL = pd.DataFrame(columns = ['Overview_Url'])

def make_clickable(link):
    # target _blank to open new window
    # extract clickable text to display for your link
    text = link.split('=')[1]
    return f'<a target="_blank" href="{link}">{text}</a>'

def replace(url):
    html = replace(url)
    soup = BeautifulSoup(html, 'html.parser')
    return soup


driver.get('https://www.premierleague.com/stats/top/players/goals?se=79')
time.sleep(2)
hovermen = driver.find_element_by_css_selector('div[aria-labelledby="dd-FOOTBALL_COMPSEASON"]')
driver.execute_script("arguments[0].click();", hovermen)
seasons = hovermen.find_element_by_xpath("//li[contains(@data-option-name, 'All')]")
driver.execute_script("arguments[0].click();", seasons)
for i in range(1,124):  
  time.sleep(0.7)
  tr = driver.find_elements_by_xpath('//table/tbody[@class= "statsTableContainer"]/tr/td[2]/a')
  c = 0
  for y in tr:
    o = []
    if c < 7:
          title = y.get_attribute('href')  
          o.append(title)
          c += 1

    elif c == 7: 
      element = wait.until(EC.presence_of_element_located((By.XPATH, '//table/tbody[@class= "statsTableContainer"]/tr[11]')))
      desired_y = (element.size['height']/2) + element.location['y']
      current_y = (driver.execute_script('return window.innerHeight') / 2) + driver.execute_script('return window.pageYOffset')
      scroll_y_by = desired_y - current_y
      driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)
      title = y.get_attribute('href')  
      o.append(title)
      c += 1
    elif 7 < c <= 20:
      title = y.get_attribute('href')  
      o.append(title)
      c += 1
    f_series = pd.Series(o, index = FL.columns)
    FL = FL.append(f_series, ignore_index = True)

  cookie = driver.find_element_by_css_selector('div[class = "btn-primary cookies-notice-accept"]')
  driver.execute_script("arguments[0].click();", cookie)
  pagenext = driver.find_element_by_css_selector('div[class = "paginationBtn paginationNextContainer"]')
  driver.execute_script("arguments[0].click();", pagenext)

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.width',1000)                   
FL['Stats Page'] = FL['Overview_Url'].str.split('/overview', n = 1, expand = True)[0].astype(str) + '/stats'
FL = FL.style.format({'Stats Page': make_clickable})
writer = pd.ExcelWriter('URL_Links.xlsx', engine='xlsxwriter')
FL.to_excel(writer, sheet_name = 'URL_Links.xlsx', index = False)
writer.save() 



def Ov(*url):
    global BaseInfo
    for ov in Overview:
        ou = urllib.request.urlopen(ov).read().decode('UTF-8')
        t  = BeautifulSoup(ou)
        try:
            cl = t.find('div', attrs =  {'data-script':'pl_player'})
            cl2 = cl.find('tbody')
            
        except:
            continue
        name = t.find('div', attrs = {'class': 'name t-colour'}).text# Player Name
        club = cl2.findAll('tr','table')
        clublist = [] 
        for cols in club:
            ele = cols.findAll('td')
            previous_m = None
            for ele in cols: 
                if ele.findAllNext('class' == 'season') or ele.findAllNext('class' == 'long'):
                    o = ele.findNext('p').text #Seasons
                    l = ele.findNext('a').text.strip()
                    m = l.split('\n')[0] #Team
                    if re.match('[1,2][0,9][0-9][0-9]/[1,2][0,9][0-9][0-9]',o):
                        if m == previous_m:
                            continue                
                        else:                                    
                            previous_m = m
                            clublist.append(name)
                            clublist.append(o)
                            clublist.append(m)
                            nl = [clublist[i:i + 3] for i in range(0, len(clublist), 3)]
                            for i in nl:   
                                x = []
                                x.append(i[0])
                                x.append(i[1])
                                x.append(i[2])
                            b_series = pd.Series(x, index = BaseInfo.columns)
                            BaseInfo = BaseInfo.append(b_series, ignore_index= True)
                    else:
                        break
                        
           
def Position(*url):
    global Positionset
    for stats_page in Slist:
        clist = []
        driver.get(stats_page) 
        time.sleep(2)
        try:
            f = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@class = 'name t-colour']"))).get_attribute("innerHTML")
            clist.append(f) #Player Name
        except:
            clist.append('Null Value')
        try:
            z = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "info"))).get_attribute('innerHTML') 
            clist.append(z) #Position
        except:
            clist.append('Null Value')
        c_series = pd.Series(clist, index = Positionset.columns)
        Positionset = Positionset.append(c_series, ignore_index = True)

         
def Stats(*url):
    global attackset
    global defenceset
    for stats_page in Slist:
        driver.get(stats_page)
        try:
            test = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'statsListBlock')))
        except:
            driver.quit()
        y = test.find_element_by_xpath("//div[@class = 'headerStat']")
        b = y.text
        if b == 'Attack':
            time.sleep(2)
            try:
                hovermen = driver.find_element_by_css_selector('div[aria-labelledby="dd-compSeasons"]')
                driver.execute_script("arguments[0].click();", hovermen)
                options = hovermen.find_elements_by_xpath("//li[contains(@data-option-name, '/')]")
            except:
                driver.quit()
            for option in options:
                alist = []
                g = option.text
                if g != None:
                    driver.execute_script("arguments[0].click();", option)
                    time.sleep(0.7)
                    try:
                        name = driver.find_element_by_xpath("//*[@class = 'name t-colour']").get_attribute("innerHTML")
                        alist.append(name) #Name
                    except:
                        alist.append('Null Name')
                    try:
                        season = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@aria-labelledby="dd-compSeasons"]'))).get_attribute('innerHTML')
                        alist.append(season) #Season
                    except:
                        alist.append('Null Value Season')
                    c = 0
                    loop = test.find_elements_by_xpath("//span[starts-with(@class, 'allStatContainer')]")
                    for x in loop:
                        h = x.text 
                        if 3 < c <= 15:
                            if h != None:
                                alist.append(h) #Stats
                            else:
                                alist.append('Null')
                            c += 1
                        else:
                            c += 1
                            continue
                    a_series = pd.Series(alist, index = attackset.columns)
                    attackset = attackset.append(a_series, ignore_index = True)
                else:
                    continue 
        elif b == 'Defence':
            time.sleep(2)
            try:
                hovermen = driver.find_element_by_css_selector('div[aria-labelledby="dd-compSeasons"]')
                driver.execute_script("arguments[0].click();", hovermen)
                options = hovermen.find_elements_by_xpath("//li[contains(@data-option-name, '/')]")
            except:
                driver.quit()
            for option in options:
                dlist = []
                g = option.text
                if g != None:
                    driver.execute_script("arguments[0].click();", option)
                    time.sleep(0.7)
                    try:
                        name = driver.find_element_by_xpath("//*[@class = 'name t-colour']").get_attribute("innerHTML")
                        dlist.append(name) #Name
                    except:
                        dlist.append('Null Name')
                    try:
                        season = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@aria-labelledby="dd-compSeasons"]'))).get_attribute('innerHTML')
                        dlist.append(season) #Season
                    except:
                        dlist.append('Null Value Season')
                    c = 0
                    loop = test.find_elements_by_xpath("//span[starts-with(@class, 'allStatContainer')]")
                    for x in loop:
                        h = x.text 
                        if 3 < c <= 21:
                            if h != None:
                                dlist.append(h) #Stats
                            else:
                                dlist.append('Null')
                            c += 1
                        else:
                            c += 1
                            continue
                    d_series = pd.Series(dlist, index = defenceset.columns)
                    defenceset = defenceset.append(d_series, ignore_index = True)
                else:
                    continue 
   
                                           

links = pd.read_excel(r'C:\Users\marsh\OneDrive\Documents\Python Scripts\URL_Links.xlsx')
Overview = links['Overview_Url'].tolist()
Slist = links['Stats Page'].tolist()
Stats(Slist)
Ov(Overview)
writer1 = pd.ExcelWriter('AttackData.xlsx', engine='xlsxwriter')
writer2 = pd.ExcelWriter('DefenceData.xlsx', engine='xlsxwriter')
writer3 = pd.ExcelWriter('PositionData.xlsx', engine='xlsxwriter')
writer4 = pd.ExcelWriter('BasicInfo.xlsx', engine='xlsxwriter')
attackset.to_excel(writer1, sheet_name = "AttackData.xlsx", index = False)
defenceset.to_excel(writer2, sheet_name = "DefenceData.xlsx", index = False)
Positionset.to_excel(writer3, sheet_name = "PositionData.xlsx", index = False)
BaseInfo.to_excel(writer4, sheet_name = 'BasicInfo.xlsx', index = False )
writer1.save()
writer2.save()
writer3.save()
writer4.save()